In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

Retrive data

In [ ]:
import spacy
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("redditADHD2012_preprocessed.csv")

print(df.head)

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

POS Tagging

In [ ]:
import re

# Modify the pos_tag_text function to return only the POS tags, not the tokens
def pos_tag_text(text):
    # Process the text through spaCy
    doc = nlp(text)
    # Extract and return only the POS tags
    return [token.pos_ for token in doc]

# Define a function to clean a single POS tag by removing special characters
def clean_pos_tag(tag):
    # This regex replaces any character that is not a letter or number with an underscore
    return re.sub(r'[^A-Za-z0-9]+', '_', tag)

# Apply the pos_tag_text function to get the POS tags
df['pos_tags'] = df['combined_text'].apply(pos_tag_text)

# Clean each POS tag using a list comprehension
df['pos_tags'] = df['pos_tags'].apply(lambda tags: [clean_pos_tag(tag) for tag in tags])


POS tags for each label of post

In [ ]:
from collections import Counter

categories = df['label'].unique()
pos_counts = {category: Counter() for category in categories}

for category in categories:
    # Subset the dataframe for the category
    subset = df[df['label'] == category]
    # Flatten the list of lists of POS tags and count each POS tag
    pos_counts[category].update([pos for sublist in subset['pos_tags'] for pos in sublist])

# Calculate the average number of each POS tag per tweet for each category
pos_averages = {category: {pos: count / len(df[df['label'] == category]) for pos, count in pos_counts[category].items()} for category in categories}

# Now we'll create a dataframe for plotting
pos_df = pd.DataFrame(pos_averages)

Diagram

In [ ]:
colors = ["#7eb0d5", "#fd7f6f", "#b2e061"]  # Pastel colors

ax = pos_df.plot(kind='bar', figsize=(12, 8), color=colors)
ax.set_xlabel('POS Tags', fontsize=14)
ax.set_ylabel('Average Number of Tags per Post', fontsize=14)
ax.legend(labels=['Neutral', 'Self-diagnosis', 'Self-medication'], fontsize=12)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
